# Temporal Convolutional Networks
Are a network architecture described in the paper ‘An Empirical Evaluation of Generic Convolutional and Recurrent Networks for Sequence Modeling’
In this notebook we demonstrate an application of this model on the adding problem, in an incremental setting. This is an adaptation of an experiment done in the original paper (The original model and adding experiment can be seen in the authors repository here: https://github.com/locuslab/TCN)

### Reference
Bai, Shaojie, J. Zico Kolter, and Vladlen Koltun. ‘An Empirical Evaluation of Generic Convolutional and Recurrent Networks for Sequence Modeling’. ArXiv:1803.01271 [Cs], 19 April 2018. http://arxiv.org/abs/1803.01271.


## Implementation
Define the data generator as written in https://github.com/locuslab/TCN/blob/2f8c2b817050206397458dfd1f5a25ce8a32fe65/TCN/adding_problem/utils.py#L6

In [2]:
from sail.models.torch.tcn import TCNRegressor
import numpy as np
import torch
from torch.autograd import Variable

def data_generator(N, seq_length):
    """
    Args:
        seq_length: Length of the adding problem data
        N: # of data in the set
    """
    X_num = torch.rand([N, 1, seq_length])
    X_mask = torch.zeros([N, 1, seq_length])
    Y = torch.zeros([N, 1])
    for i in range(N):
        positions = np.random.choice(seq_length, size=2, replace=False)
        X_mask[i, 0, positions[0]] = 1
        X_mask[i, 0, positions[1]] = 1
        Y[i,0] = X_num[i, 0, positions[0]] + X_num[i, 0, positions[1]]
    X = torch.cat((X_num, X_mask), dim=1)
    return Variable(X), Variable(Y)

Define parameters. 
Everything is using same default values as original experiment. The only difference is the original experiment is batch learned, where we are incremental.
Original experiment ran for 10 epochs of 50k samples. We run incrementally for 100k samples.

In [3]:
input_channels = 2
n_classes = 1
batch_size = 30
seq_length = 400
#epochs = 10
X_train, Y_train = data_generator(100000, seq_length)
X_test, Y_test = data_generator(1000, seq_length)


In [4]:
# Note: We use a very simple setting here (assuming all levels have the same # of channels.
channel_sizes = [30]*8 
kernel_size = 7
dropout = 0.0

learning_rate =4e-3

tcn = TCNRegressor(input_channels,n_classes, layers=channel_sizes, ks=kernel_size, conv_dropout=dropout, fc_dropout=dropout, batch_size=batch_size)
tcn.initialize()
from torchinfo import summary
summary(tcn.module_,X_train[:batch_size].shape,col_names=("input_size","output_size","num_params"))

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
_TCN                                     [30, 2, 400]              [30, 1]                   --
├─_TemporalConvNet: 1-1                  [30, 2, 400]              [30, 30, 400]             --
│    └─Sequential: 2-1                   [30, 2, 400]              [30, 30, 400]             --
│    │    └─_TemporalBlock: 3-1          [30, 2, 400]              [30, 30, 400]             6,930
│    │    └─_TemporalBlock: 3-2          [30, 30, 400]             [30, 30, 400]             12,720
│    │    └─_TemporalBlock: 3-3          [30, 30, 400]             [30, 30, 400]             12,720
│    │    └─_TemporalBlock: 3-4          [30, 30, 400]             [30, 30, 400]             12,720
│    │    └─_TemporalBlock: 3-5          [30, 30, 400]             [30, 30, 400]             12,720
│    │    └─_TemporalBlock: 3-6          [30, 30, 400]             [30, 30, 400]             12,720
│    │    └─

In [5]:

#optimizer = torch.optim.Adam(tcn.get_params(), lr=learning_rate)
y_pred= []

for i in range(0,X_train.size(0), batch_size):
    if i + batch_size > X_train.size(0):
        x, y = X_train[i:], Y_train[i:]
    else:
        x, y = X_train[i:(i+batch_size)], Y_train[i:(i+batch_size)]
    tcn.partial_fit(x, y)
    y_pred.append(tcn.predict(x)[0][0]) 

  epoch    train_loss     dur
-------  ------------  ------
      1        1.1807  0.3524
      2        0.6894  0.2986
      3        0.1889  0.2893
      4        0.1907  0.2817
      5        0.2289  0.2767
      6        0.1686  0.2668
      7        0.2370  0.2688
      8        0.1691  0.2690
      9        0.3127  0.2751
     10        0.2317  0.2742
     11        0.2157  0.2851
     12        0.2544  0.2825
     13        0.1575  0.2706
     14        0.1449  0.2734
     15        0.2908  0.2836
     16        0.2702  0.2730
     17        0.1157  0.3574
     18        0.1722  0.2762
     19        0.2336  0.2716
     20        0.2199  0.2924
     21        0.1084  0.2787
     22        0.1115  0.2756
     23        0.2308  0.2749
     24        0.2115  0.2876
     25        0.1957  0.2737
     26        0.2063  0.2696
     27        0.1170  0.2794
     28        0.2333  0.2854
     29        0.1311  0.2877
     30        0.2262  0.2817
     31        0.1673  0.3140
     32   

In [ ]:
print(f"loss: {tcn.get_loss(y_pred=tcn.predict(X_test), y_true=Y_test)}")

TypeError: 'int' object is not callable

In [ ]:

import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'vscode'
y_true = y.flatten().tolist()
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(x.size(0))), y=y_pred, name='predicted',
                         line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=list(range(x.size(0))), y=y_true, name='actual',
                         line=dict(color='blue', width=2)))
fig.show()

KeyboardInterrupt: 